# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix (q=0,ω=0) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.235506897958                   -0.50    7.0
  2   -7.250338827517       -1.83       -1.39    1.0
  3   -7.251222450358       -3.05       -1.97    4.0
  4   -7.251190287290   +   -4.49       -2.04    4.0
  5   -7.251317779217       -3.89       -2.50    2.0
  6   -7.251337484011       -4.71       -3.05    2.0
  7   -7.251338769537       -5.89       -3.95    1.0
  8   -7.251338792132       -7.65       -4.13    4.0
  9   -7.251338798389       -8.20       -4.72    2.0
 10   -7.251338798650       -9.58       -5.22    3.0
 11   -7.251338798698      -10.32       -5.60    3.0
 12   -7.251338798704      -11.24       -6.07    2.0
 13   -7.251338798704      -12.29       -6.70    3.0
 14   -7.251338798705      -13.27       -7.25    2.0
 15   -7.251338798705      -14.45       -7.75    2.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Apply ε† = 1 - χ0 (vc + fxc)

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end

eps_fun (generic function with 1 method)

eager diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.04282023838409628
[ Info: Arnoldi iteration step 2: normres = 0.5296348738113354
[ Info: Arnoldi iteration step 3: normres = 0.9958801423758608
[ Info: Arnoldi iteration step 4: normres = 0.2841523188038295
[ Info: Arnoldi iteration step 5: normres = 0.39879892136135797
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.35e-02, 5.09e-02, 3.21e-01, 2.27e-01, 3.20e-02)
[ Info: Arnoldi iteration step 6: normres = 0.3949381630917344
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.18e-02, 2.22e-01, 2.97e-01, 1.05e-01, 4.57e-02)
[ Info: Arnoldi iteration step 7: normres = 0.1167849073430397
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (7.31e-04, 3.47e-02, 1.46e-02, 7.14e-02, 7.82e-02)
[ Info: Arnoldi iteration step 8: normres = 0.09841018256800493
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (3.19e-0